# Average 3 Fold predictions

In [80]:
import json
import numpy as np
from pathlib import Path
import pandas as pd

fold_paths = [
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTCPP/SAM-H/CPP-Net-Setting/Fold-1-LR",
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTCPP/SAM-H/CPP-Net-Setting/Fold-2-LR",
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTCPP/SAM-H/CPP-Net-Setting/Fold-3-LR"
]

In [81]:
detection_results = []
cell_pq_results = []
tissue_metrics = []
global_f1 = []
global_prec = []
global_rec = []


for fold, fold_path in enumerate(fold_paths):
    json_path = Path(fold_path) / "inference_results.json"
    
    with open(json_path, 'r') as file:
        data = json.load(file)
    detection_results.append(pd.DataFrame.from_dict(data["nuclei_metrics_d"]))
    cell_pq_results.append(pd.DataFrame(data["nuclei_metrics_pq"], index=[0]))
    tissue_metrics.append(pd.DataFrame.from_dict(data["tissue_metrics"]))
    global_f1.append(float(data["dataset"]["f1_detection"]))
    global_prec.append(data["dataset"]["precision_detection"])
    global_rec.append(data["dataset"]["recall_detection"])

detection_avg = pd.concat([each.stack() for each in detection_results],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()
cell_pq_avg = pd.concat([each.stack() for each in cell_pq_results],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()           
tissue_metrics_avg = pd.concat([each.stack() for each in tissue_metrics],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()
             
global_f1 = np.mean(global_f1)
global_prec = np.mean(global_prec)
global_rec = np.mean(global_rec)

### Detection-Scores

In [82]:
detection_avg.T

,f1_cell,prec_cell,rec_cell
Neoplastic,0.695703,0.723855,0.670290
Inflammatory,0.582357,0.615900,0.552282
Connective,0.524128,0.547554,0.502704
Dead,0.175197,0.268581,0.140462
Epithelial,0.704357,0.733527,0.678224


In [83]:
print(f"f1: {global_f1}")
print(f"prec: {global_prec}")
print(f"rec: {global_rec}")

f1: 0.8179755839642903
prec: 0.8626492133067755
rec: 0.7777569126384346


### Cell-Type-PQ-Scores

In [84]:
cell_pq_avg

,Neoplastic,Inflammatory,Connective,Dead,Epithelial
0,0.565854,0.393275,0.389609,0.069133,0.547857


### Tissue-Type-PQ-Scores

In [85]:
tissue_metrics_avg.T

,Dice,Jaccard,mPQ,bPQ
adrenal_gland,0.829459,0.726831,0.481371,0.687048
bile-duct,0.779334,0.712400,0.448318,0.652870
bladder,0.785134,0.732105,0.572140,0.685692
breast,0.819227,0.713145,0.494972,0.657931
cervix,0.720321,0.728072,0.477740,0.672026
colon,0.699370,0.648711,0.419095,0.566670
esophagus,0.823998,0.720436,0.522501,0.649727
headneck,0.664246,0.651436,0.465266,0.623078
kidney,0.789046,0.736374,0.516703,0.685961
liver,0.852092,0.754249,0.489525,0.705794


In [86]:
row_mean = tissue_metrics_avg.mean(axis=1)
row_mean

Dice       0.793089
Jaccard    0.710805
mPQ        0.468443
bPQ        0.655224
dtype: float64

In [87]:
row_mean = tissue_metrics_avg.std(axis=1)
row_mean

Dice       0.052442
Jaccard    0.033348
mPQ        0.048651
bPQ        0.035420
dtype: float64